# IPO Information Crawler

In [1]:
import selenium
from selenium import webdriver
import time
import pandas as pd
from pandas import DataFrame
from glob import glob
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

## BS4 Crawlering

In [2]:
def ipo_information(url):
    c_information = pd.read_html(url)
    if len(c_information)  != 17:
        ipo_name = c_information[16][1][0]
        market = c_information[16][1][1]
        category = c_information[16][1][2]
        sales = c_information[16][1][7]
        net_profit = c_information[16][1][8]
        capital = c_information[16][3][8]
        code = int(c_information[16][3][1])
        b_class = c_information[16][3][3]
        offical_price = c_information[17][1][3]
        bank = c_information[17][1][4]
        if c_information[18][0][0] == '인수회사':
            predict_demand = c_information[19][2][0]
            part_offering = c_information[19][2][1]
            ipo_date = c_information[19][2][5]
            c_r_o_i = c_information[19][1][8]
            lock_up = c_information[19][1][8]
        else:
            predict_demand = c_information[18][2][0]
            part_offering = c_information[18][2][1]
            ipo_date = c_information[18][2][5]
            c_r_o_i = c_information[18][1][8]
            lock_up = c_information[18][1][8]
        lst = [ipo_name,market,category,sales,net_profit,capital,code,b_class,offical_price,bank,predict_demand,part_offering,ipo_date,c_r_o_i,lock_up]
        return lst
    else:
        pass

def bs_stock(url):
    return pd.read_html(url)[14][1:21][["기업명","공모가(원)","첫날종가(원)"]]

In [3]:
# IPO 기본정보 추합 & stock data

IPO_basic_information = pd.DataFrame()
firstday_return = pd.DataFrame()

# Crawling
for page_num in tqdm(range(1,49)):
    basic_url = f'http://www.38.co.kr/html/fund/index.htm?o=k&page={page_num}'
    response = requests.get(basic_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    p_soup = soup.select_one('tbody').select('td')
    for idx in range(6,182,7):
        a = str(p_soup[idx])
        p_url = 'http://www.38.co.kr/html/fund/?o=v&no={}&l=&page={}'.format(a[a.find('no')+3:a.find('no')+7],page_num)
        p_df = pd.DataFrame(ipo_information(p_url)).T
        IPO_basic_information = IPO_basic_information.append(p_df)
for page_num in tqdm(range(1,49)):
    stock_url = f'http://www.38.co.kr/html/fund/index.htm?o=nw&page={page_num}'
    firstday_return = firstday_return.append(bs_stock(stock_url))

100%|█████████████████████████████████████████████| 48/48 [00:11<00:00,  4.15it/s]


## DataFrame 전처리

In [4]:
IPO_basic_information = IPO_basic_information.rename(columns= {0:'기업명',1:'시장구분',2:'업종',3:'매출액',
                                                               4:'순이익',5:'자본금',6:'종목코드',
                                                               7:'기업구분',8:'확정공모가',9:'주간사',
                                                               10:'수요예측일',11:'공모청약일',12:'상장일',
                                                               13:'기관경쟁률',14:'의무보유확약'})

In [5]:
firstday_return = firstday_return.rename(columns={'공모가(원)':'공모가', '첫날종가(원)':'첫날종가'})

In [6]:
# 상장 안한 기업 제거

firstday_return = firstday_return.iloc[5:]
IPO_basic_information = IPO_basic_information.iloc[3:]

In [7]:
def Series_preprocessing(series):
    series = series.apply(lambda x: x[:x.find('(')])
    series = series.apply(lambda x: x.replace(',','') if (x.strip() not in '-')  else '0')
    series = series.apply(lambda x: int(x.replace('.','')))
    return series

In [8]:
def reset_index(df):
    df = df.reset_index()
    del df['index']
    return df
def astype_str(series):
    series = series.apply(lambda x: str(x))
    return series
def astype_int(series):
    series = series.apply(lambda x: int(x))
    return series

In [9]:
def company_filtering(df):
    df = df[df['기업명'].str.contains('스팩') == False]
    df['기업명'] = df['기업명'].apply(lambda x: x.replace('(유가)',''))
    df = df.reset_index()
    del df['index']
    return df

def org_pre_processing(series):
    series = series.apply(lambda x: x[x.find('률')+1:x.find('의')].strip()[:-2])
    return series

def org_pre_processing(series):
    series = series.apply(lambda x: x[x.find('률')+1:x.find('의')].strip()[:-2])
    return series

def lockup_pre_processing(series):
    series = series.apply(lambda x: x[x.find('약')+1:].strip()[:-1])
    return series

def offering_price_pre_processing(series):
    series = series.apply(lambda x: int(x[:-1].replace(',','')))
    return series

In [10]:
firstday_return = reset_index(firstday_return)
IPO_basic_information = reset_index(IPO_basic_information)

In [11]:
IPO_basic_information['매출액'] = Series_preprocessing(IPO_basic_information['매출액'])
IPO_basic_information['순이익'] = Series_preprocessing(IPO_basic_information['순이익'])
IPO_basic_information['자본금'] = Series_preprocessing(IPO_basic_information['자본금'])

In [12]:
IPO_basic_information = company_filtering(IPO_basic_information)

In [13]:
firstday_return = company_filtering(firstday_return)

In [14]:
IPO_basic_information.head()

,기업명,시장구분,업종,매출액,순이익,자본금,종목코드,기업구분,확정공모가,주간사,수요예측일,공모청약일,상장일,기관경쟁률,의무보유확약
0,바이오노트,거래소,의료용품 및 기타 의약 관련제품 제조업,622355,671556,1266407,377740,중소일반,"9,000 원",NH투자증권,2022.12.08 ~ 2022.12.09,2022.12.13 ~ 2022.12.14,2022.12.22,기관경쟁률 3.29:1 의무보유확약 2.77%,기관경쟁률 3.29:1 의무보유확약 2.77%
1,에스에이엠지엔터테인먼트,코스닥,"영화, 비디오물, 방송프로그램 제작 및 배급업",38374,-11087,4295,419530,중소일반,"17,000 원",NH투자증권,2022.11.17 ~ 2022.11.18,2022.11.24 ~ 2022.11.25,2022.12.06,기관경쟁률 44.73:1 의무보유확약 0.05%,기관경쟁률 44.73:1 의무보유확약 0.05%
2,펨트론,코스닥,기타 전기장비 제조업,57871,4499,5322,168360,중소일반,"8,000 원",하나증권,2022.11.08 ~ 2022.11.09,2022.11.15 ~ 2022.11.16,2022.11.24,기관경쟁률 129.06:1 의무보유확약 11.41%,기관경쟁률 129.06:1 의무보유확약 11.41%
3,인벤티지랩,코스닥,의약품 제조업,1862,-9271,4102,389470,중소일반,"12,000 원",한국투자증권,2022.11.08 ~ 2022.11.09,2022.11.11 ~ 2022.11.14,2022.11.22,기관경쟁률 14.37:1 의무보유확약 0.00%,기관경쟁률 14.37:1 의무보유확약 0.00%
4,엔젯,코스닥,평판디스플레이 제조용 기계 제조업,10055,-29867,5258,419080,중소일반,"10,000 원",미래에셋증권,2022.11.03 ~ 2022.11.04,2022.11.09 ~ 2022.11.10,2022.11.18,기관경쟁률 42.15:1 의무보유확약 6.00%,기관경쟁률 42.15:1 의무보유확약 6.00%


In [ ]:
for ind in IPO_basic_information.query('기업명.str.contains("구.")').index.to_list():
    l = IPO_basic_information['기업명'][ind].find('(구.')+3
    IPO_basic_information['기업명'][ind] = IPO_basic_information['기업명'][ind][l:-1]

In [15]:
IPO_basic_information['기관경쟁률'] = org_pre_processing(IPO_basic_information['기관경쟁률'])
IPO_basic_information['의무보유확약'] = lockup_pre_processing(IPO_basic_information['의무보유확약'])

In [16]:
IPO_basic_information['확정공모가'] = offering_price_pre_processing(IPO_basic_information['확정공모가'])

In [17]:
IPO_basic_information.head()

,기업명,시장구분,업종,매출액,순이익,자본금,종목코드,기업구분,확정공모가,주간사,수요예측일,공모청약일,상장일,기관경쟁률,의무보유확약
0,바이오노트,거래소,의료용품 및 기타 의약 관련제품 제조업,622355,671556,1266407,377740,중소일반,9000,NH투자증권,2022.12.08 ~ 2022.12.09,2022.12.13 ~ 2022.12.14,2022.12.22,3.29,2.77
1,에스에이엠지엔터테인먼트,코스닥,"영화, 비디오물, 방송프로그램 제작 및 배급업",38374,-11087,4295,419530,중소일반,17000,NH투자증권,2022.11.17 ~ 2022.11.18,2022.11.24 ~ 2022.11.25,2022.12.06,44.73,0.05
2,펨트론,코스닥,기타 전기장비 제조업,57871,4499,5322,168360,중소일반,8000,하나증권,2022.11.08 ~ 2022.11.09,2022.11.15 ~ 2022.11.16,2022.11.24,129.06,11.41
3,인벤티지랩,코스닥,의약품 제조업,1862,-9271,4102,389470,중소일반,12000,한국투자증권,2022.11.08 ~ 2022.11.09,2022.11.11 ~ 2022.11.14,2022.11.22,14.37,0.00
4,엔젯,코스닥,평판디스플레이 제조용 기계 제조업,10055,-29867,5258,419080,중소일반,10000,미래에셋증권,2022.11.03 ~ 2022.11.04,2022.11.09 ~ 2022.11.10,2022.11.18,42.15,6.00


In [18]:
IPO_basic_information.to_csv('../data/IPO_basic_information.csv',encoding='cp949')

In [19]:
firstday_return.to_csv('../data/firstday_return.csv',encoding='cp949')